Lets see whether simple mt5 model overfits in small data samples

In [2]:
from transformers import (AutoTokenizer,
                          AutoModelForSeq2SeqLM,
                          Seq2SeqTrainer,
                          Seq2SeqTrainingArguments,
                          DataCollatorForSeq2Seq
                          )
from datasets import load_dataset
import evaluate
import numpy as np
import torch
import warnings
import wandb
warnings.filterwarnings("ignore")
import random
# Set all seeds for reproducibility
random.seed(100)
np.random.seed(100)
torch.manual_seed(100)
torch.cuda.manual_seed_all(100)
# Load aryal's dataset from hf
ds = load_dataset("sumitaryal/nepali_grammatical_error_correction")

c:\Users\Lenovo\Desktop\Nepali_GEC\nepali_gec\myenv\Lib\site-packages\pydantic\_internal\_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
c:\Users\Lenovo\Desktop\Nepali_GEC\nepali_gec\myenv\Lib\site-packages\pydantic\_internal\_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have 

In [3]:
ds

DatasetDict({
    train: Dataset({
        features: ['incorrect_sentence', 'correct_sentence'],
        num_rows: 7723971
    })
    valid: Dataset({
        features: ['incorrect_sentence', 'correct_sentence'],
        num_rows: 406525
    })
})

In [4]:
# select randomly few samples from train 
# split further into train and valid dataset
small_dataset = ds["train"].shuffle(seed=42).select(range(100))
small_dataset = small_dataset.train_test_split(test_size=0.1, seed=42)
small_dataset["valid"] = small_dataset["test"] # Rename the split in the DatasetDict
del small_dataset["test"]
small_dataset

DatasetDict({
    train: Dataset({
        features: ['incorrect_sentence', 'correct_sentence'],
        num_rows: 90
    })
    valid: Dataset({
        features: ['incorrect_sentence', 'correct_sentence'],
        num_rows: 10
    })
})

In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_id = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [6]:
prefix = "वाक्य सच्याउनुहोस्: "

def preprocess(batch):
    
    inputs = [prefix + inp for inp in batch["incorrect_sentence"]]

    # tokenize input (incorrect)
    input_encodings = tokenizer(
        inputs, 
        max_length=128,
        truncation=True 
    )
    # tokenize target (correct)
    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(
            batch["correct_sentence"], 
            max_length=128,
            truncation=True
        )

    # set labels for seq2seq training                           # for seq2deq models, the "labels" are the token IDs of the target sequence
    input_encodings["labels"] = target_encodings["input_ids"]   

    return input_encodings

dataset_encoded = small_dataset.map(preprocess, batched=True) 


Map: 100%|██████████| 10/10 [00:00<00:00, 427.84 examples/s]


In [7]:
#pytorch model expects in tensor format
dataset_encoded.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

In [8]:
import evaluate
import numpy as np
from collections import Counter
import numpy as np
import re

def tokenize_nepali(text):
    """Tokenizes Nepali text: splits on spaces and removes punctuation."""
    # Remove punctuation commonly used in Nepali
    text = re.sub(r"[।,!?]", "", text)
    return text.strip().split()

def gleu_sentence(reference, prediction, max_n=4):
    """
    Compute sentence-level GEC-GLEU.
    Returns a score between 0 and 1.
    """
    ref_tokens = tokenize_nepali(reference)
    hyp_tokens = tokenize_nepali(prediction)
    
    # Adjust max_n for short sentences
    max_n = min(max_n, len(ref_tokens), len(hyp_tokens))
    if max_n == 0:
        return 0.0  # empty sentence
    
    scores = []
    for n in range(1, max_n+1):
        ref_ngrams = Counter([tuple(ref_tokens[i:i+n]) for i in range(len(ref_tokens)-n+1)])
        hyp_ngrams = Counter([tuple(hyp_tokens[i:i+n]) for i in range(len(hyp_tokens)-n+1)])
        overlap = sum((ref_ngrams & hyp_ngrams).values())
        precision = overlap / max(1, sum(hyp_ngrams.values()))
        recall = overlap / max(1, sum(ref_ngrams.values()))
        scores.append(min(precision, recall))
    return sum(scores) / max_n

def corpus_gec_gleu(references, predictions):
    """
    Compute corpus-level GEC-GLEU.
    `references` can be a list of strings or a list of single-item lists.
    """
    # Flatten single-reference lists
    refs_flat = [r[0] if isinstance(r, list) else r for r in references]
    
    scores = [gleu_sentence(r, p) for r, p in zip(refs_flat, predictions)]
    return float(np.mean(scores))

# Load metrics once
bleu_metric = evaluate.load("sacrebleu")
chrf_metric = evaluate.load("chrf")
bertscore_metric = evaluate.load("bertscore")

def compute_metrics(eval_pred):
    """
    Compute BLEU, chrF, Correction Accuracy, and BERTScore for Nepali GEC.
    Handles both token IDs and plain text predictions.
    """
    predictions, labels = eval_pred

    # --- Handle tuple outputs (e.g., logits + labels) ---
    if isinstance(predictions, tuple):
        predictions = predictions[0]

    # --- If preds/labels are lists of strings, skip decoding ---
    if isinstance(predictions[0], str) and isinstance(labels[0], str):
        preds_clean = [p.strip() for p in predictions]
        refs_clean = [r.strip() for r in labels]
    else:
        # Convert to numpy arrays
        predictions = np.array(predictions)
        labels = np.array(labels)

        # Handle logits (vocab dimension)
        if predictions.ndim == 3:
            predictions = predictions.argmax(axis=-1)

        # Replace -100 with pad_token_id
        predictions = np.where(predictions == -100, tokenizer.pad_token_id, predictions)
        labels = np.where(labels == -100, tokenizer.pad_token_id, labels)

        # Decode
        preds = tokenizer.batch_decode(predictions, skip_special_tokens=True, clean_up_tokenization_spaces=True)
        refs = tokenizer.batch_decode(labels, skip_special_tokens=True, clean_up_tokenization_spaces=True)

        preds_clean = [p.strip() for p in preds]
        refs_clean = [r.strip() for r in refs]

    # --- Format for metrics ---
    references = [[r] for r in refs_clean]
    metrics = {}

    # --- BLEU ---
    try:
        non_empty_indices = [i for i, (p, r) in enumerate(zip(preds_clean, refs_clean)) if p and r]
        if non_empty_indices:
            preds_bleu = [preds_clean[i] for i in non_empty_indices]
            refs_bleu = [[refs_clean[i]] for i in non_empty_indices]
            bleu_result = bleu_metric.compute(predictions=preds_bleu, references=refs_bleu)
            metrics["bleu"] = bleu_result["score"]
        else:
            metrics["bleu"] = 0.0
    except Exception as e:
        print(f"BLEU computation failed: {e}")
        metrics["bleu"] = 0.0

    # --- chrF ---
    try:
        chrf_result = chrf_metric.compute(predictions=preds_clean, references=refs_clean)
        metrics["chrf"] = chrf_result["score"]
    except Exception as e:
        print(f"chrF computation failed: {e}")
        metrics["chrf"] = 0.0

    # --- Correction Accuracy ---
    try:
        exact_matches = np.mean([p == r for p, r in zip(preds_clean, refs_clean)])
        metrics["correction_accuracy"] = exact_matches
    except Exception as e:
        print(f"Correction accuracy computation failed: {e}")
        metrics["correction_accuracy"] = 0.0

    # # --- BERTScore ---
    # try:
    #     non_empty_indices_bert = [i for i, (p, r) in enumerate(zip(preds_clean, refs_clean)) if p and r]
    #     if non_empty_indices_bert:
    #         preds_bert = [preds_clean[i] for i in non_empty_indices_bert]
    #         refs_bert = [refs_clean[i] for i in non_empty_indices_bert]
    #         bertscore_result = bertscore_metric.compute(
    #             predictions=preds_bert,
    #             references=refs_bert,
    #             lang="ne",
    #             model_type="microsoft/mdeberta-v3-base"
    #         )
    #         metrics["bertscore_f1"] = float(np.mean(bertscore_result["f1"]))
    #     else:
    #         metrics["bertscore_f1"] = 0.0
    # except Exception as e:
    #     print(f"BERTScore computation failed: {e}")
    #     metrics["bertscore_f1"] = 0.0
        
    # --- GLEU (SacreBLEU) ---
    try:


        gleu_score = corpus_gec_gleu(refs_clean, preds_clean)


        metrics["gleu"] = gleu_score
    except Exception as e:
        print("GLEU failed:", e)
        metrics["gleu"] = 0.0

    # --- Print one sample for sanity ---
    if len(preds_clean) > 0:
        print(f"🔍 Sample - Pred: '{preds_clean[0][:50]}...' | Ref: '{refs_clean[0][:50]}...' | Match: {preds_clean[0] == refs_clean[0]}")

    return metrics

preds = ["मेरो नाम सन्तोष हो ।", "म स्कुल जान्छु ।", "म खाना खान्छु ।"]
refs  = ["मेरो नाम सन्तोष हो ।", "म स्कुल जान्छु ।", "म खाना खान्छु ।"]
compute_metrics((preds, refs))

🔍 Sample - Pred: 'मेरो नाम सन्तोष हो ।...' | Ref: 'मेरो नाम सन्तोष हो ।...' | Match: True


{'bleu': 100.00000000000004,
 'chrf': 100.0,
 'correction_accuracy': np.float64(1.0),
 'gleu': 1.0}

In [9]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

In [10]:
from transformers import AutoModelForSeq2SeqLM, BitsAndBytesConfig
quantization_config = BitsAndBytesConfig(load_in_8bit=True,
                                        llm_int8_threshold=6.0,  
                                        llm_int8_has_fp16_weight=False )
model = AutoModelForSeq2SeqLM.from_pretrained(model_id,
                                              quantization_config=quantization_config,
                                              device_map="auto")

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training
# from peft import unload
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q", "k", "v", "o"],
    lora_dropout=0.05, # disable for overfit test
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)
model = prepare_model_for_kbit_training(model)
# model.gradient_checkpointing_enable()
# model = unload(model)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()
# model.config.use_cache = False  # Required for gradient checkpointing


trainable params: 1,376,256 || all params: 301,553,024 || trainable%: 0.4564


In [12]:
seq2seq_data_collator = DataCollatorForSeq2Seq(
    tokenizer, 
    # model=model,
    pad_to_multiple_of=8,  # Optional: for better performance
    return_tensors="pt", 
    padding=True)


In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, EarlyStoppingCallback, TrainerCallback
from math import ceil
import os
import wandb

wandb.finish()
wandb.init(project="nepali-grammar-correction", name="mt5-nepali")
run_id = wandb.run.id

batch_size = 4
num_train_epochs = 3
gradient_accumulation_steps = 2
learning_rate = 1e-4
weight_decay = 0.01
lr_scheduler_type = "linear"
steps_per_epoch = ceil(len(dataset_encoded["train"]) // (batch_size * gradient_accumulation_steps))    # no. of steps per epoch # log once per epoch

eval_steps = max(1, steps_per_epoch) // 2           # Log 2 times per epoch
num_training_steps = steps_per_epoch * num_train_epochs
warmup_steps = int(0.05 * num_training_steps)


model_name = f"{model_id}-finetuned-gec"

# Create directories
os.makedirs(f"../outputs/checkpoints/{model_name}", exist_ok=True)
os.makedirs("../outputs/best_model", exist_ok=True)
os.makedirs("../outputs/logs", exist_ok=True)

training_args = Seq2SeqTrainingArguments(output_dir=f"../outputs/checkpoints/{model_name}",
                                         num_train_epochs=num_train_epochs,

                                         # Memory Optimization:
                                         per_device_train_batch_size=batch_size,
                                         per_device_eval_batch_size= 2 * batch_size,
                                         gradient_accumulation_steps=gradient_accumulation_steps,  # Simulate larger batch size eg: 8 * 2 = 16
                                         fp16=False,                                                # Use mixed precision if GPU supports it
                                         dataloader_pin_memory=True,                        # ✅ Faster data loading
                                         dataloader_num_workers=4,                          # ✅ Parallel data loading

                                         gradient_checkpointing=False,                      # ✅ Disable for speed
                                         
                                         # Logging & Saving:
                                         logging_steps=1,           # log the training loss and metrics every X steps
                                         eval_strategy="epoch",          # performs evaluation per epoch
                                        #  eval_steps=eval_steps,
                                         save_strategy="epoch",          # saves model checkpoint per epoch
                                        #  save_steps=230000,
                                         save_total_limit=2,             # keep last 2 checkpoints for safety
                                         overwrite_output_dir=True,      # Overwrite previous runs

                                         # Best Model saving:
                                         load_best_model_at_end=True,        # Load the best model at the end
                                         metric_for_best_model="chrf",   # Use chrf to determine best model
                                         greater_is_better=True,            # larger chrf is better

                                         # performance
                                         warmup_steps=warmup_steps,             # Gradually increases LR at start
                                         learning_rate=learning_rate,
                                         weight_decay=weight_decay,             # L2 regularization
                                         lr_scheduler_type=lr_scheduler_type,
                                         max_grad_norm=1.0,                     # Prevent exploding gradients
                                         optim="paged_adamw_8bit",              # Better optimizer for quantized models

                                         # Seq2seq specific:
                                         predict_with_generate=True,    # essential for seq2seq , If not set then metrics will be computed on meaningless logits
                                         generation_max_length=64,      # Max output length
                                         generation_num_beams=4,        # 1=greedy, 4=beam search (slower but better)

                                         report_to="wandb",          # This enables automatic logging
                                         run_name=f"{run_id}",
                                         push_to_hub=True,                       # save the model to HF
                                         hub_model_id="AIsumit123/Nepali_GEC_mt5_model",
                                         seed=42,
                                         data_seed=42,
                                         )






wandb: Currently logged in as: lsumit008 (lsumit008-khwopa-college-of-engineering) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: WARNING Fatal error while uploading data. Some run data will not be synced, but it will still be written to disk. Use `wandb sync` at the end of the run to try uploading.


In [28]:
import torch
from transformers import TrainerCallback
import wandb

class SamplePredictionCallback(TrainerCallback):
    """Generate predictions on a few validation samples and log to W&B."""

    def __init__(self, tokenizer, eval_dataset, num_samples=5, max_length=64):
        self.tokenizer = tokenizer
        self.eval_dataset = eval_dataset
        self.num_samples = num_samples
        self.max_length = max_length

    @torch.no_grad()
    def on_evaluate(self, args, state, control, **kwargs):
        model = kwargs["model"]
        model.eval()
        device = model.device

        # Select sample rows
        samples = self.eval_dataset.select(range(self.num_samples))

        table = wandb.Table(columns=["Input", "Target", "Prediction"])

        for sample in samples:

            # Use the original raw text fields, not token IDs
            inp_text = sample["incorrect_sentence"]
            tgt_text = sample["correct_sentence"]

            # Tokenize individual sample
            tokenized = self.tokenizer(
                inp_text,
                return_tensors="pt",
                truncation=True,
                padding=False
            ).to(device)

            # Safe generate (LoRA-friendly)
            with torch.cuda.amp.autocast(enabled=False):
                output_ids = model.generate(
                    input_ids=tokenized["input_ids"],
                    attention_mask=tokenized["attention_mask"],
                    max_length=self.max_length,
                    num_beams=1
                )

            pred_text = self.tokenizer.decode(output_ids[0], skip_special_tokens=True)

            table.add_data(inp_text, tgt_text, pred_text)

            del tokenized, output_ids
            torch.cuda.empty_cache()

        wandb.log({"sample_predictions": table})


In [ ]:
callbacks = [
    SamplePredictionCallback(tokenizer, eval_dataset=small_dataset["valid"], num_samples=5, max_length = 64)
]
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset_encoded["train"],
    eval_dataset=dataset_encoded["valid"],  
    tokenizer=tokenizer,
    data_collator=seq2seq_data_collator,
    compute_metrics=compute_metrics,
    callbacks=callbacks
      
)



In [30]:
# Complete safety check
def safe_training_check(trainer):
    """Comprehensive pre-training safety check"""
    print(" Running pre-training safety checks...")

    # 1. Check model is on correct device
    print(f"Model device: {next(trainer.model.parameters()).device}")

    # 2. Check dataset sizes
    print(f"Train dataset size: {len(trainer.train_dataset)}")
    print(f"Eval dataset size: {len(trainer.eval_dataset)}")

    # 3. Test data loading
    try:
        sample_batch = next(iter(trainer.get_train_dataloader()))
        print(" Data loading works")
        # print(f"Batch keys: {sample_batch.keys()}")
    except Exception as e:
        print(f" Data loading failed: {e}")
        return False

    # 4. Test evaluation
    try:
        trainer.model.eval()    # Set to evaluation mode
        print(" Performing evaluation check...")
        eval_results = trainer.evaluate()
        print(" Evaluation successful")
        print(f"Initial metrics: {eval_results}")
        return True
    except Exception as e:
        print(f" Evaluation failed: {e}")
        return False

# Usage
if safe_training_check(trainer):
    print("\n" + "="*60)
    print("✅ All checks passed! Starting training...")
    print("="*60)
    trainer.train()
    print("✅ Training complete!")

else:
    print(" Fix issues before training!")

 Running pre-training safety checks...
Model device: cuda:0
Train dataset size: 90
Eval dataset size: 10
 Data loading works
 Performing evaluation check...


🔍 Sample - Pred: '<extra_id_0> ।...' | Ref: 'अहिले कार्यक्रममा हेर्नुभयो भने वडाध्यक्ष प्रमुख अ...' | Match: False
 Evaluation successful
Initial metrics: {'eval_loss': 10.177413940429688, 'eval_model_preparation_time': 0.0019, 'eval_bleu': 0.4093082060797297, 'eval_chrf': 1.086395705048532, 'eval_correction_accuracy': 0.0, 'eval_gleu': 0.0020833333333333333, 'eval_runtime': 17.5206, 'eval_samples_per_second': 0.571, 'eval_steps_per_second': 0.114}

✅ All checks passed! Starting training...


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Epoch,Training Loss,Validation Loss,Model Preparation Time,Bleu,Chrf,Correction Accuracy,Gleu
1,31.635600,9.772861,0.001900,0.415208,1.570373,0.000000,0.002083
2,14.641400,8.902305,0.001900,0.410058,2.723920,0.000000,0.007986
3,28.399900,9.016022,0.001900,0.412558,2.592565,0.000000,0.007292


🔍 Sample - Pred: '<extra_id_0> ।...' | Ref: 'अहिले कार्यक्रममा हेर्नुभयो भने वडाध्यक्ष प्रमुख अ...' | Match: False
🔍 Sample - Pred: '<extra_id_0> ।...' | Ref: 'अहिले कार्यक्रममा हेर्नुभयो भने वडाध्यक्ष प्रमुख अ...' | Match: False
🔍 Sample - Pred: '<extra_id_0> ।...' | Ref: 'अहिले कार्यक्रममा हेर्नुभयो भने वडाध्यक्ष प्रमुख अ...' | Match: False
✅ Training complete!


In [52]:
# This will pick the latest checkpoint and train
# trainer.train(resume_from_checkpoint=True)

In [31]:
import gc
import torch

# del model       # or del comet_model
gc.collect()
torch.cuda.empty_cache()

## Inference

In [ ]:
from transformers import AutoTokenizer
import torch

base = "google/mt5-small"
lora = "../outputs/best_model"

tokenizer = AutoTokenizer.from_pretrained(base)

# Load base + LoRA
from transformers import AutoModelForSeq2SeqLM
from peft import PeftModel

model = AutoModelForSeq2SeqLM.from_pretrained(base)
model = PeftModel.from_pretrained(model, lora)
model.eval()

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

def predict(text):
    input_text = f"वाक्य सुधार्नुहोस्: {text}"
    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        truncation=True,
        padding=True
    ).to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=64,
            num_beams=4
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

test_sentence = "नगरपालिका कस्तो किसिमको पर्यटक ल्याउन सक्छे "
corrected = predict(test_sentence)
print(f"Original: {test_sentence}")
print(f"Corrected: {corrected}")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Original: नगरपालिका कस्तो किसिमको पर्यटक ल्याउन सक्छे 
Corrected: <extra_id_0> ?


In [48]:
print(predict("यो एक गलत वाक्य हो ।"))


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


<extra_id_0> हो ।


In [ ]:
def correct_batch(texts, batch_size=8):
    """
    Correct grammar for multiple sentences
    """
    corrected_texts = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        
        # Add prefix to each text
        input_texts = [f"वाक्य सुधार्नुहोस्: {text}" for text in batch_texts]
        
        
    
        # Tokenize
        inputs = tokenizer(
            input_texts,
            return_tensors = "pt",
            truncation = True,
            padding=True
        ).to(device)
        
        # Generate correction
        with torch.no_grad():
            outputs = model.generate(
                input_ids=inputs.input_ids,
                attention_mask=inputs.attention_mask,
                max_length=64,
                num_beams=5,
                repetition_penalty=2.5
            )
            
        # Decode batch
        batch_corrected = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        corrected_texts.extend(batch_corrected)
        
    return corrected_texts
        
    
test_sentences = small_dataset["train"]["incorrect_sentence"][:]
labels = small_dataset["train"]["correct_sentence"][:]
corrected_sentences = correct_batch(test_sentences)
for orig, corr, lab in zip(test_sentences, corrected_sentences, labels):
    print(f"Original:  {orig}")
    print(f"Corrected: {corr}")
    print(f"label:     {lab}")
    print("---")

Original:  गोपनीयता सुरक्षामा उच्च सतर्कता अपनाउन र मातहतका निकायको बफादारीको खाँचो रहेको विज्ञहरु औल्याउँछन् ।
Corrected: <extra_id_0> ।
label:     गोपनीयता सुरक्षामा उच्च सतर्कता अपनाउन सरकार र मातहतका निकायको बफादारीको खाँचो रहेको विज्ञहरु औल्याउँछन् ।
---
Original:  ग्रिन विन (विदेश निर्यात गर्ने) कफी विदेशमा प्रतिकिलो १० देखी १५ डलरसम्ममा बिक्री हुन्छ ।
Corrected: <extra_id_0> ।
label:     ग्रिन विन (विदेश निर्यात गर्ने) कफी विदेशमा प्रतिकिलो १० देखि १५ डलरसम्ममा बिक्री हुन्छ ।
---
Original:  उपत्यकाबाहिर मेडिकल कलेज खोल्न र राजधानीबाहिरका मनसायपत्र (एलओआई) पाएर पूर्वाधार पुरा गरेका मेडिकल कलेजलाई रोक्न नहुने उनको धारणा छ ।
Corrected: <extra_id_0> छ ।
label:     उपत्यकाबाहिर मेडिकल कलेज खोल्न र राजधानीबाहिरका मनसायपत्र (एलओआई) पाएर पूर्वाधार पूरा गरेका मेडिकल कलेजलाई रोक्न नहुने उनको धारणा छ ।
---
Original:  तर, प्राधिकरणले डेढ अर्ब मात्रै उपलब्ध प्रतिबद्धता जनाएको उनले बताए ।
Corrected: <extra_id_0> ।
label:     तर, प्राधिकरणले डेढ अर्ब मात्रै उपलब्ध गराउने प्रतिबद्धता जनाएको उनल

In [51]:
compute_metrics((corrected_sentences, labels))

🔍 Sample - Pred: '<extra_id_0> ।...' | Ref: 'गोपनीयता सुरक्षामा उच्च सतर्कता अपनाउन सरकार र मात...' | Match: False


{'bleu': 0.8975998136046814,
 'chrf': 2.607411569860353,
 'correction_accuracy': np.float64(0.0),
 'gleu': 0.013655577945223627}